In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.NRMS import NRMSModel

In [7]:
hparams = {
    'scale':'demo',
    'name':'nrms',
    'mode':'train',
    'batch_size':5,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'value_dim':16,
    'head_num':16,
    'epochs':10,
    'metrics':'auc,group_auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:0',
    'attrs': ['title'],
    'k':-1,
    'select':None,
    'save_step':0,
    'save_each_epoch':False,
    'train_embedding':False,
    'news_id':False,
    'validate':False
}

In [8]:
device = torch.device(hparams['device'])
vocab, loaders = prepare(hparams)

In [ ]:
nrmsModel = NRMSModel(vocab=vocab,hparams=hparams).to(device)

In [40]:
train(nrmsModel, hparams, loaders)

TypeError: object of type 'MIND_iter' has no len()